# CESI HumanForYou

L'entreprise de produits pharmaceutiques HumanForYou basée en Inde emploie environ 4000 personnes. Cependant, chaque année elle subit un turn-over d'environ 15% de ses employés nécessitant de retrouver des profils similaires sur le marché de l'emploi.

La direction trouve que ce niveau de turn-over n'est pas bon pour l'entreprise car :

* Les projets sur lesquels étaient les employés quittant la société prennent du retard ce qui nuit à la réputation de l'entreprise auprès de ses clients et partenaires.

* Un service de ressources humaines de taille conséquente doit être conservé car il faut avoir les moyens de trouver les nouvelles recrues.

* Du temps est perdu à l'arrivée des nouveaux employés car ils doivent très souvent être formés et ont besoin de temps pour devenir pleinement opérationnels dans leur nouvel environnement.

Le direction fait donc appel à notre équipe, spécialistes de l'analyse de données, pour déterminer les facteurs ayant le plus d'influence sur ce taux de turn-over et lui proposer des modèles afin d'avoir des pistes d'amélioration pour donner à leurs employés l'envie de rester.

### Table des matières

1. [Préparation de l'environnement](#chapter1)
    1. [Importation des librairies](#section_1_1)
    2. [Importation des données](#Section_1_2)
2. [Visualisation des données](#chapter2)
    1. [Données du service des ressources humaines](#section_2_1)
    2. [Dernière évaluation du manager](#section_2_2)
    3. [Enquête qualité de vie au travail](#section_2_3)
    4. [Horaires de travail](#section_2_4)
3. [Transformation des données](#chapter3)
    1. [Calcul des durées de travail](#section_3_1)
    2. [Concaténation des données](#section_3_2)
    3. [Ajout de valeur](#section_3_3)
    4. [Suppression colonne](#section_3_4)
    5. [Normalisation](#section_3_5)
    6. [Standardisation](#section_3_6)
    7. [Création des jeux de données ](#section_3_7)
4. [Analyses statistiques](#chapter4)
    1. [Analyse des données entrantes](#section_4_1)
    2. [Analyse de l'attrition](#section_4_2)
5. [Algorithmes](#chapter5)
6. [Evaluation des modèles](#chapter6)


<a id="chapter1"></a>
## Préparation de l'environnement

<a id="section_1_1"></a>
### Importation des librairies

Tout d'abord, nous devons importer toutes les bibliothèques que nous utiliserons.

In [ ]:
# imports
import numpy as np
import os
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.random import default_rng
from datetime import datetime
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier

<a id="section_1_2"></a>
### Importation des données

Les données utilisées pour nos analyses proviennent de fichier CSV depuis Github et doivent être charger dans nos variables.

In [ ]:
#Importation des données depuis Github 
general_url = "https://raw.githubusercontent.com/anthonylorendeaux/CESI-IA-CSV/master/general_data.csv"
manager_url = "https://raw.githubusercontent.com/anthonylorendeaux/CESI-IA-CSV/master/manager_survey_data.csv"
employee_url = "https://raw.githubusercontent.com/anthonylorendeaux/CESI-IA-CSV/master/employee_survey_data.csv"
in_time_url = "https://raw.githubusercontent.com/anthonylorendeaux/CESI-IA-CSV/master/in_time.csv"
out_time_url = "https://raw.githubusercontent.com/anthonylorendeaux/CESI-IA-CSV/master/out_time.csv"

#Lecture des csv
general_info_data = pd.read_csv(general_url)
manager_survey_data = pd.read_csv(manager_url)
employee_survey_data = pd.read_csv(employee_url)
in_time_data = pd.read_csv(in_time_url)
out_time_data = pd.read_csv(out_time_url)

<a id="chapter2"></a>
## Visualisation des données

Dans un premier temps, nous regardons toutes les données que nous avons. Un certain nombre de données concernant les employés nous a donc été transmis par le service des ressources humaines ainsi que par des fiches d'évaluation.

Les données ont été anonymisées : un employé de l'entreprise sera représenté par le même EmployeeID dans l'ensemble des fichiers qui suivent.

<a id="section_2_1"></a>
### Données du service des ressources humaines

Pour chaque employé, le service des ressources humaines vous confie les informations en sa possession :

* Age : L'âge de l'employé en 2015.
* Attrition : L'objet de notre étude, est-ce que l'employé a quitté l'entreprise durant l'année 2016 ?

* BusinessTravel : A quel fréquence l'employé a été amené à se déplacer dans le cadre de son travail en 2015 ? (Non-Travel = jamais, Travel_Rarely= rarement, Travel_Frequently = fréquemment)

* DistanceFromHome : Distance en km entre le logement de l'employé et l'entreprise.

* Education : Niveau d'étude : 1=Avant College (équivalent niveau Bac), 2=College (équivalent Bac+2), 3=Bachelor (Bac+3), 4=Master (Bac+5) et 5=PhD (Thèse de doctorat).

* EducationField : Domaine d'étude, matière principale

* EmployeeCount : booléen à 1 si l'employé était compté dans les effectifs en 2015.

* EmployeeId : l'identifiant d'un employé

* Gender : Sexe de l'employé

* JobLevel : Niveau hiérarchique dans l'entreprise de 1 à 5

* JobRole : Métier dans l'entreprise

* MaritalStatus : Statut marital du salarié (Célibataire, Marié ou Divorcé).

* MonthlyIncome : Salaire brut en roupies par mois

* NumCompaniesWorked : Nombre d'entreprises pour lequel le salarié a travaillé avant de rejoindre HumanForYou.

* Over18 : Est-ce que le salarié a plus de 18 ans ou non ?

* PercentSalaryHike : % d'augmentation du salaire en 2015.

* StandardHours : Nombre d'heures par jour dans le contrat du salarié.

* StockOptionLevel : Niveau d'investissement en actions de l'entreprise par le salarié.

* TotalWorkingYears : Nombre d'années d'expérience en entreprise du salarié pour le même type de poste.

* TrainingTimesLastYear : Nombre de jours de formation en 2015

* YearsAtCompany : Ancienneté dans l'entreprise

* YearsSinceLastPromotion : Nombre d'années depuis la dernière augmentation individuelle

* YearsWithCurrentManager : Nombre d'années de collaboration sous la responsabilité du manager actuel de l'employé.



In [ ]:
general_info_data.head(5)

Il est aussi intéressant de connaitre le type des variables qui composent le fichier.

In [ ]:
print('Shape of general_info_data :',general_info_data.shape)
general_info_data.info()

<a id="section_2_2"></a>
### Dernière évaluation du manager

Ce fichier contient la dernière évaluation de chaque employé faite pas son manager en février 2015.

Il contient les données suivantes :

* L'identifiant de l'employé : EmployeeID

* Une évaluation de son implication dans son travail notée 1 ('Faible'), 2 ("Moyenne"), 3 ("Importante") ou 4 ("Très importante") : JobInvolvement

* Une évaluation de son niveau de performance annuel pour l'entreprise notée 1 ("Faible"), 2 ("Bon"), 3 ("Excellent") ou 4 ("Au delà des attentes") : PerformanceRating

In [ ]:
manager_survey_data.head(3)

Type des données :

In [ ]:
manager_survey_data.info()

<a id="section_2_3"></a>
### Enquête qualité de vie au travail

Ce fichier provient d'une enquête soumise aux employés en juin 2015 par le service RH pour avoir un retour concernant leur qualité de vie au travail.

Une organisation avait été mise en place pour que chacun puisse répondre à ce questionnaire sur son lieu de travail en concertation avec les managers mais il n'y avait pas d'obligation.

Les employés devaient répondre à 3 questions sur le niveau de satisfaction concernant :

* L'environnement de travail, noté 1 ("Faible"), 2 ("Moyen"), 3 ("Élevé") ou 4 ("Très élevé") : EnvironmentSatisfaction

* Son travail, noté de 1 à 4 comme précédemment : JobSatisfaction

* Son équilibre entre vie professionnelle et vie privée, noté 1 ("Mauvais"), 2 ("Satisfaisant"), 3 ("Très satisfaisant") ou 4 ("Excellent") : WorkLifeBalance

Lorsque un employé n'a pas répondu à une question, le texte "NA" apparaît à la place de la note.

In [ ]:
employee_survey_data.head()

Type des données :

In [ ]:
employee_survey_data.info()

<a id="section_2_4"></a>
### Horaires de travail

Des badgeuses sont installées et utilisées dans l'entreprise depuis quelques années. Il a été jugé opportun par la direction de nous transmettre les horaires d'entrée et de sortie des employés sur une période de l'année choisie représentative d'une activité moyenne pour l'ensemble des services.

Nous avons donc 2 fichiers traçants les horaires d'arrivée à leur poste et de départ de leur poste de l'ensemble des employés par date sur une période allant du 1er janvier au 31 décembre 2015.

Données d'arrivée des employées:

In [ ]:
in_time_data.head(2)

Type des données :

In [ ]:
in_time_data.info()

Données de départ des employées: 

In [ ]:
out_time_data.head(2)

Type des données :

In [ ]:
out_time_data.info()

<a id="chapter3"></a>
## Transformation des données

Plusieurs données ne peuvent pas être exploiter en l'état, il faut donc trier et retravailler les données.

<a id="section_3_1"></a>
### Calcul des durées de travail

Les données de temps ne sont pas exploitables sous cette forme, il faut donc les tranformer.

Avoir des heures d'entrées et de sortie de nos employés n'est pas très significatifs, c'est pour cela que nous remplaçons toutes les valeurs par la moyenne de temps de travail de chaque employé.

Cependant : 
* Les dates sont stockées en tant que chaine de caractère et il est compliqué de les exploiter.
* Cetaines données valent "NaN", ce qui veut dire qu'un employé a été absent au travail

Pour remédier à ça, nous transformons les données en objet Datetime. De plus lorsqu'un employé est absent au travail, son temps moyen de travail est de 0 donc nous remplaçons les NaN par 0.

Avant de mettre en place nos changements, nous devons renommer la colonne (sans nom) qui correspond aux IDs des employés. Cette actions est prise puisque dans le csv nous avons le même nombre de ligne que sur les autres csv.

In [ ]:
# Renommage des colonnes sans nom de nos csv in et out
in_time_data.rename(columns={"Unnamed: 0": "EmployeeID"}, inplace=True)
in_time_data.set_index('EmployeeID', inplace=True)
in_time_data
out_time_data.rename(columns={"Unnamed: 0": "EmployeeID"}, inplace=True)
out_time_data.set_index('EmployeeID', inplace=True)

# Suppression des colonnesoù l'employée est absent (valeur NaN)
in_time_data=in_time_data.dropna(axis=1,how='all')
out_time_data=out_time_data.dropna(axis=1,how='all')

# Remplacement des NaN par 0
in_time_data.fillna(0, inplace=True)
out_time_data.fillna(0, inplace=True)

out_time_data.head(3)

On transforme nos chaines de caractère en objets datetime.

In [ ]:
for date in in_time_data.columns:
    in_time_data[date]=pd.to_datetime(in_time_data[date])
    out_time_data[date]=pd.to_datetime(out_time_data[date])

On calcule dans un nouveau dataset le nombre d'heure passé qu'en employé passe au travail par jour.

In [ ]:
time_work_per_day=pd.DataFrame()

cols=in_time_data.columns
for col in cols:
    time_work_per_day[col]=((out_time_data[col] - in_time_data[col]).dt.total_seconds() /3600)

time_work_per_day.head()

Nous ajoutons ensuite une colonne représentant : 
* La moyenne de temps passsé au travail par employé sur l'année 2015 
* Le nombre d'absences au travail par employé durant l'année 2015 

Les autres colonnes sont ensuite supprimées.

In [ ]:
time_work_per_day['MeanTimeWorkOverYear2015']=round(time_work_per_day.astype(int).mean(axis=1),2)
time_work_per_day['absences_par_jour']=(time_work_per_day == 0).astype(int).sum(axis=1)
time_work_per_day = time_work_per_day.drop((time_work_per_day.columns[0:-2]), axis = 1)
time_work_per_day.head()

<a id="section_3_2"></a>
### Concaténation des données

Pour la suite des analyses, nous allons rassembler toutes les données sur une même variable. Comme sur chaque csv, l'ID des employées est inscrit, il est facile de concater les données.

In [ ]:
concat_time_csv = general_info_data.merge(time_work_per_day, on='EmployeeID')
concat_manager_csv = concat_time_csv.merge(manager_survey_data, on='EmployeeID')
temp_concat = concat_manager_csv.merge(employee_survey_data, on='EmployeeID')
temp_concat = temp_concat.set_index('EmployeeID')
temp_concat.info()

A l'aide des informations précédentes, nous remarquons plusieurs choses:
* Tous les champs ne possèdent pas le même nombre de tuples
* Les champs ayants un type objet corréspondent à des variables qualitatives

Pour la suite des analyses, nous devons d'abord harmoniser nos données pour ne plus avoir les deux remarques précédentes.

<a id="section_3_3"></a>
### Ajout de valeur

Pour palier au manque de certaines données, nous comblons les valeurs manquantes par la valeur médiane de ses champs.


In [ ]:
# Nombre de champs avec des valeurs vides
final_data = temp_concat.copy()
final_data[final_data.columns[final_data.isnull().any()]].isnull().sum()

In [ ]:
# Ajout des valeurs médianes
final_data.fillna(round(final_data.median()),inplace=True)
final_data.info()

<a id="section_3_4"></a>
### Suppression de colonne

Il est important de vérifier qu'il n'y est pas des champs avec valeur similaire partout. Cela signifit que l'information n'est pas pertinente et qu'elle peut être supprimer.


In [ ]:
final_data.describe(include='all').T

A l'aide de ce tableau, nous remarquons qu'il existe des champs à valeur unique :
- EmployeeCount : la valeur min et max est égale à 1
- Over18 : cette varible possède une unique valeur (unique = 1)
- StandardHours : la valeur min et max est égale à 8

Nous supprimons donc ces champs pour la suite des travaux.

In [ ]:
final_data.drop(['EmployeeCount', 'Over18', 'StandardHours'], axis=1, inplace = True)
final_data.head(5)

<a id="section_3_5"></a>
### Normalisation

Comme dans la suite de ce projet nous devrons utiliser des algorithmes de machine learning, il est important de prendre en compte les recommandations de ses derniers. Comme de nombreux algorithmes d'apprentissage automatique ne peuvent pas fonctionner directement sur des données qualitatives, nous devons prévoir un ensemble de données dont toutes les variables d'entrée et les variables de sortie soient numériques.

Afin de n'avoir que des données quantitatives, nous utilisons le One-hot Encoding. Quand une variables n'est pas ordinale, cette solution va créer des variables supplémentaires dans le jeu de donnée pour représenter chacune des catégories.

Les champs concernés sont : "BusinessTravel","Department","EducationField", "JobRole","MaritalStatus", "Gender".
Et vont être remplacés par : 
* BusinessTravel_Non-Travel
* BusinessTravel_Travel_Frequently
* BusinessTravel_Travel_Rarely
* Department_Human Resources
* Department_Research & Development
* Department_Sales
* EducationField_Human Resources
* EducationField_Life Sciences
* EducationField_Marketing
* EducationField_Medical
* EducationField_Other
* EducationField_Technical Degree
* JobRole_Healthcare Representative
* JobRole_Human Resources
* JobRole_Manufacturing Director	
* JobRole_Research Director	
* JobRole_Research Scientist	
* JobRole_Sales Executive	
* JobRole_Sales Representative	
* MaritalStatus_Divorced	
* MaritalStatus_Married	
* MaritalStatus_Single	
* Gender_Female	
* Gender_Male

In [ ]:
# One-hot Encoding
final_data2 = pd.get_dummies(final_data, 
prefix=["BusinessTravel","Department","EducationField", "JobRole","MaritalStatus", "Gender"],
columns=["BusinessTravel","Department","EducationField", "JobRole","MaritalStatus", "Gender"])
final_data2.head(5)

<a id="section_3_6"></a>

### Standardisation

In [ ]:
# Suppression de la colonne Attrition
final_data3 = final_data2.drop("Attrition",axis=1)

# Normalisation des données en utilisant le z-score 
norm = StandardScaler().fit_transform(final_data3)
final_data3 = pd.DataFrame(norm, columns=final_data3.columns)
final_data3

<a id="section_3_7"></a>

### Création des jeux de données


In [ ]:
# Définitions de nos variables X et y
X = final_data3
y = final_data2["Attrition"]

seed =42
test_size = 0.2
# Fixer un seed pour avoir les mêmes résultats à chaque essai
np.random.seed(seed)

# Séparer les données d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=test_size)

A la suite de cette section, nous avons deux ensembles de données nettoyés et complets:
* final_data :  données qualitatives et quantitatives
* final_data2 : données uniquement quantitatives 
* final_data3 : données uniquement quantitatives et standardisées 


<a id="chapter4"></a>

## Analyses statistiques

Nous séparons nos données en deux parties : 
-  Les valeurs entrantes qui correspondent à tous les paramêtres qui possèdent différentes valeurs (Ex: DistanceFromHome, Education, Age ...)
- La valeur de sortie (variable 'Attrition') qui permet de savoir si l'employée à quitté ou non l'entreprise l'année suivante.

Pour cela, nous allons analyser nos deux groupes.

<a id="section_4_1"></a>
### Analyste des données entrantes

#### Histogramme 

Concernant les données quantitatives, nous affichons nos valeurs de façon visuelle afin d'avoir un meilleur appercu de l'allure de nos données.

In [ ]:
final_data.hist(bins=50, figsize=(20,15))
plt.show()

#### Corrélation des données

Maintenant nous voulons voir si nos valeurs possèdent des liens entre-elles. Pour cela nous utilisons une matrice de corrélation. Elle permet d'évaluer la dépendence entre plusieurs variables en même temps. Le résultat est une table contenant les coefficients de corrélation entre chaque variable et les autres.


In [ ]:
# Matrice de corrélation
corr_matrix = final_data2.corr(method="pearson")

plt.figure(figsize=(40,30))
sns.heatmap(corr_matrix, annot= True, cbar=True, cmap="Blues_r")
plt.show()

corr_matrix = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
corr_matrix = corr_matrix.unstack().reset_index()
corr_matrix.columns = ["feature1", "feature2", "Correlation"]
corr_matrix.dropna(subset = ['Correlation'], inplace = True)
corr_matrix['Correlation'] = round(corr_matrix['Correlation'], 2)
corr_matrix['Correlation'] = abs(corr_matrix['Correlation'])
corr_matrix = corr_matrix.sort_values(by = 'Correlation', ascending = False)
value_high_corr = corr_matrix[corr_matrix['Correlation']>0.7]

value_high_corr

In [ ]:
# Suppression cols
cols_pas_ethic = ["Gender_Male", "Gender_Female", "YearsWithCurrManager", "PerformanceRating", "Age", "MaritalStatus_Single", "MaritalStatus_Married", "MaritalStatus_Divorced"]
cols_big_corr = ["Department_Sales", "BusinessTravel_Travel_Rarely"]

final_data4 = final_data3.drop(cols_pas_ethic,axis=1)
final_data4 = final_data4.drop(cols_big_corr,axis=1)
final_data4.describe().T

In [ ]:
final_data4["MonthlyIncome"] = np.log(final_data4["MonthlyIncome"])
final_data4["MonthlyIncome"].hist()

<a id="section_4_2"></a>

### Analyse de l'attrition

Maintenant que nous avons analyser les paramêtres indépendament de l'attrition, nous allons regarder leur influence sur cette dernière.

#### Courbes de densité superposées

A l'aide des courbes de densité superposées, nous regardons la répartition des valeurs quantitatives pour chaque variables en fonction de l'attrition. 

In [ ]:
def area_chart(df,feature):
    ax=sns.kdeplot(df[df['Attrition']=='Yes'][feature],
             shade=True,label='Attrition = Yes')
    ax=sns.kdeplot(df[df['Attrition']=='No'][feature],
                 shade=True,label='Attrition = No')
    ax.legend()

cols = ['DistanceFromHome','MonthlyIncome','TotalWorkingYears','TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion','PercentSalaryHike','YearsWithCurrManager','MeanTimeWorkOverYear2015','absences_par_jour', "Age"]

plt.figure(figsize=(40,30))
for i, col in enumerate(cols):
    ax = plt.subplot(4, 3, i+1)
    area_chart(final_data, col)
    # for bar in plots.patches:
    #     percentage = '{:.1f}%'.format(100 * bar.get_height() / len(final_data[col]))
    #     x = bar.get_x() + bar.get_width() / 2
    #     y = bar.get_height()
    #     plots.annotate(percentage, (x, y),ha='center',va='center',size=15, xytext=(0, 3),
    #                textcoords='offset points')
plt.tight_layout(pad=3.0)
plt.show()

Cependant comme nous voyons peu de différence à l'oeil nu, nous allons dans un deuxième temps des boites à moustache.

#### Boîtes à moustache

A l'aide de boites à moustache, nous regardons la répartition des valeurs quantitatives pour chaque variables en fonction de l'attrition.

In [ ]:
cols = ['DistanceFromHome','MonthlyIncome','TotalWorkingYears','TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion','PercentSalaryHike','YearsWithCurrManager','MeanTimeWorkOverYear2015','absences_par_jour']

plot_rows=2
plot_cols=5
fig = make_subplots(rows=plot_rows, cols=plot_cols,subplot_titles=(cols))

col_i = 0
for i in range(1, plot_rows + 1):
    for j in range(1, plot_cols + 1):
        for t in px.box(final_data, x="Attrition", y=cols[col_i]).data:
            fig.add_trace(t,row=i, col=j)

        col_i=col_i+1
fig.update_layout(height=600, width=1200)
fig.show()

#### Histogrammes

A l'aide des histogrammes, nous regardons la répartition des valeurs qualitatives pour chaque variables en fonction de l'attrition.

In [ ]:
cols = ['BusinessTravel', 'Department', 'Education', 'EducationField', 'Gender', 'JobLevel', 'JobRole', 'MaritalStatus', 'StockOptionLevel', 'JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']

plt.figure(figsize=(40,20))
for i, col in enumerate(cols):
    ax = plt.subplot(3, 5, i+1)
    plots = sns.histplot(data=final_data, x=col, ax=ax, hue=final_data['Attrition'], multiple='dodge', shrink=0.8)


    # From axis.patches get bar lengths
    get_bars = plots.patches
    half_bar_length = int(len(get_bars)/2)
    bar_left = get_bars[:half_bar_length]
    bar_right = get_bars[half_bar_length:]
    
    # Place %employees on top of each bar
    for L, R in zip(bar_left, bar_right):
        left_height = L.get_height()
        right_height = R.get_height()
        length_total = left_height + right_height

        # place calculated employee percentage on top of each bar
        ax.text(L.get_x() + L.get_width()/2., left_height + 30, '{0:.0%}'.format(left_height/length_total), ha="center")
        ax.text(R.get_x() + R.get_width()/2., right_height + 30, '{0:.0%}'.format(right_height/length_total), ha="center")
plt.tight_layout(pad=3.0)
plt.show()

In [ ]:
cols = ['BusinessTravel', 'Department', 'Education', 'EducationField', 'Gender', 'JobLevel', 'JobRole', 'MaritalStatus', 'StockOptionLevel', 'JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']

plot_rows=7
plot_cols=2
fig = make_subplots(rows=plot_rows, cols=plot_cols,subplot_titles=(cols))

col_i = 0
for i in range(1, plot_rows + 1):
    for j in range(1, plot_cols + 1):
        for t in px.histogram(final_data, x=cols[col_i],
                 color='Attrition', barmode='group',text_auto=True,
                 height=400).data:
            fig.add_trace(t,row=i, col=j)

        col_i=col_i+1
fig.update_layout(height=1400, width=1000)
fig.show()


In [ ]:
cols = ['BusinessTravel', 'Department', 'Education', 'EducationField', 'Gender', 'JobLevel', 'JobRole', 'MaritalStatus', 'StockOptionLevel', 'JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']

attrition = final_data['Attrition'].tolist()
result = {}

for col in cols:
    current_col = final_data[col].tolist()

    #values = {name: "", [0,1]}
    values = {}
    label = []
    list_yes = []
    list_no = []

    for index in range(len(current_col)):

        try:
            i = label.index(current_col[index])
        except:
            i = -1
        
        # On ajoute
        if(i != -1):
            if(attrition[index] == 'Yes'):
                list_yes[i] += 1
            else:
                list_no[i] += 1
        # On créée
        else:
            label.append(current_col[index])
            if(attrition[index] == 'Yes'):
                list_yes.append(1)
                list_no.append(0)
            else:
                list_yes.append(0)
                list_no.append(1)
    
    pourcent_yes = []
    pourcent_no = []

    for j in range(len(list_yes)):
        no = list_no[j]
        yes = list_yes[j]
        total = no + yes
        pourcent_yes.append("{}%".format(round(100 * yes / total)))
        pourcent_no.append("{}%".format(round(100 * no / total)))
    
    plot_rows=7
    plot_cols=2
    fig = make_subplots(rows=plot_rows, cols=plot_cols,subplot_titles=(cols))

    for ii in range(1, plot_rows + 1):
        for jj in range(1, plot_cols + 1): 
            show=False
            if (ii == 1 and jj == 1):
                show=True
            fig.add_trace(go.Bar(
                                name='Yes',
                                x=label,
                                y=list_yes,
                                offsetgroup=0,
                                text = pourcent_yes,
                                marker_color="steelblue",
                                showlegend=show
                            ),row=ii, col=jj)
            fig.add_trace(go.Bar(
                                name='No',
                                x=label,
                                y=list_no,
                                offsetgroup=1,
                                text = pourcent_no,
                                marker_color="coral",
                                showlegend=show
                            ),row=ii, col=jj)
fig.update_layout(height=1800, width=1000, title="Attrition for each columns")
fig.show()

<a id="chapter5"></a>
## Algorithmes

In [ ]:
# Listes des algorithmes  
models = {"Logistic Regression": LogisticRegression(),
          "KNN": KNeighborsClassifier(),
          "Random Forest": RandomForestClassifier(),
          "Perceptron": Perceptron(),
          "Descente de gradient": SGDClassifier()}

# fonction appliquant un fit et score sur chacun des modeles
def fit_and_score(models, X_train, y_train):
    
    # Fixer un seed pour avoir les mêmes résultats à chaque essai 
    np.random.seed(42)
    
    # Dictionnaire pour sauvegarder les scores
    model_scores = {}
    for name, model in models.items():
        # Fit le modèle
        model.fit(X_train, y_train)
        # Evaluer le modèle selon le score de chaque algorithme
        model_scores[name] = (cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy')).mean()
        
    return model_scores

In [ ]:
model_scores = fit_and_score(models=models, X_train=X_train, y_train=y_train)
model_scores

In [ ]:
# Comparaison des résultats obtenus
model_compare = pd.DataFrame(model_scores, index=["accuracy"])
model_compare.T.plot.bar();

<a id="chapter5"></a>

## Evaluation des modèles
